In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

# 定义目录
data_dir = './videos/MRI515_T2'
label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

# 自动分割结果所在目录（使用 nolap 版本）
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')

# 输出 CSV 文件路径（最终长格式结果）
csv_write_path = os.path.join(data_dir, prompt_dir, prompt_dir + '_area_results_long.csv')

# 检查所需目录是否存在
if not os.path.exists(label_dir) or not os.path.exists(auto_seg_dir):
    raise FileNotFoundError("One or more required directories do not exist!")

# 每个像素对应的实际面积（例如，单位为平方毫米）
spacing_factor = 0.6875 * 0.6875  # = 0.47265625

# 定义 4 个类别对应的目标像素值
classes = [50, 100, 150, 200]

results = []

# 处理每个 label 文件
for label_file in os.listdir(label_dir):
    # 假设文件名格式为 "123.png"，提取数字部分作为索引
    try:
        label_index = int(os.path.splitext(label_file)[0])
    except ValueError:
        print(f"Skipping file with non-numeric name: {label_file}")
        continue

    # 构造 label 文件的完整路径
    label_path = os.path.join(label_dir, label_file)
    label_index = str(int(label_index))
    
    # 构造自动分割 mask 的完整路径（4 个类别）
    auto_seg_paths = [
        os.path.join(auto_seg_dir, f"frame_{label_index}_obj_{i}.png")
        for i in range(1, len(classes) + 1)
    ]
    # 检查是否所有自动分割文件均存在
    if not all(os.path.exists(p) for p in auto_seg_paths):
        print(f"Auto segmentation files missing for {label_file}, skipping.")
        continue

    # 读取 label mask
    label_mask = np.array(Image.open(label_path))

    # 读取所有自动分割 mask
    auto_masks = [np.array(Image.open(p)) for p in auto_seg_paths]

    file_result = {"file_name": label_file}

    # 对每个类别进行面积计算
    for idx, class_val in enumerate(classes, start=1):
        # label: threshold处理，选取与目标值相近的像素（例如 ±10）
        label_class_mask = (np.abs(label_mask - class_val) < 10).astype(np.uint8)
        # auto segmentation：对应 obj_i 文件，使用相同的阈值条件
        auto_class_mask = (np.abs(auto_masks[idx - 1] - class_val) < 10).astype(np.uint8)
        # 调整自动分割 mask 的尺寸以匹配 label mask
        target_size = (label_mask.shape[1], label_mask.shape[0])
        auto_class_mask = cv2.resize(auto_class_mask, target_size, interpolation=cv2.INTER_NEAREST)

        # 计算面积：非0像素数 × spacing_factor
        label_area = np.sum(label_class_mask) * spacing_factor
        auto_area  = np.sum(auto_class_mask)  * spacing_factor

        # 保存结果，键名例如 "class1_label_area", "class1_auto_area"
        file_result[f"class{idx}_label_area"] = label_area
        file_result[f"class{idx}_auto_area"] = auto_area

    results.append(file_result)
    # 打印处理结果
    print(f"Processed {label_file}:", end=" ")
    for idx in range(1, len(classes) + 1):
        print(f"Class{idx} -> Label: {file_result[f'class{idx}_label_area']:.2f}, Auto: {file_result[f'class{idx}_auto_area']:.2f}; ", end="")
    print()

# 将宽格式结果转换为长格式
rows = []
for res in results:
    pid = os.path.splitext(res["file_name"])[0]  # 使用文件名（去除扩展名）作为 PID
    for cls in range(1, len(classes) + 1):
        rows.append({
            "PID": pid,
            "Class": cls,
            "Auto Area": res[f"class{cls}_auto_area"],
            "Manual Area": res[f"class{cls}_label_area"]
        })

results_long_df = pd.DataFrame(rows)

# 保存长格式结果到 CSV
results_long_df.to_csv(csv_write_path, index=False)
print(f"Results saved to {csv_write_path}")

# 输出预览
print(results_long_df.head())


Processed 00261.png: Class1 -> Label: 1229.85, Auto: 1153.28; Class2 -> Label: 1147.14, Auto: 1147.14; Class3 -> Label: 1229.38, Auto: 1198.66; Class4 -> Label: 1206.22, Auto: 1246.39; 
Processed 00212.png: Class1 -> Label: 1400.95, Auto: 1433.57; Class2 -> Label: 911.28, Auto: 960.91; Class3 -> Label: 851.25, Auto: 881.50; Class4 -> Label: 1524.32, Auto: 1505.41; 
Processed 00378.png: Class1 -> Label: 939.64, Auto: 860.71; Class2 -> Label: 842.75, Auto: 823.37; Class3 -> Label: 921.68, Auto: 912.23; Class4 -> Label: 829.51, Auto: 735.45; 
Processed 00071.png: Class1 -> Label: 1173.61, Auto: 1160.84; Class2 -> Label: 966.58, Auto: 1022.36; Class3 -> Label: 951.46, Auto: 947.20; Class4 -> Label: 1239.30, Auto: 1286.10; 
Processed 00328.png: Class1 -> Label: 893.32, Auto: 732.62; Class2 -> Label: 554.90, Auto: 570.97; Class3 -> Label: 586.09, Auto: 557.26; Class4 -> Label: 883.39, Auto: 950.04; 
Processed 00193.png: Class1 -> Label: 1064.42, Auto: 1003.45; Class2 -> Label: 1056.39, Auto:

Processed 00180.png: Class1 -> Label: 1410.41, Auto: 1358.89; Class2 -> Label: 1169.82, Auto: 1104.12; Class3 -> Label: 1097.51, Auto: 1081.44; Class4 -> Label: 1331.95, Auto: 1336.20; 
Processed 00080.png: Class1 -> Label: 1165.10, Auto: 1082.38; Class2 -> Label: 628.16, Auto: 664.55; Class3 -> Label: 683.93, Auto: 700.95; Class4 -> Label: 1064.89, Auto: 1014.79; 
Processed 00380.png: Class1 -> Label: 555.84, Auto: 627.21; Class2 -> Label: 1149.50, Auto: 1192.04; Class3 -> Label: 1072.93, Auto: 1096.09; Class4 -> Label: 631.00, Auto: 723.16; 
Processed 00062.png: Class1 -> Label: 1149.97, Auto: 912.23; Class2 -> Label: 806.82, Auto: 750.58; Class3 -> Label: 830.93, Auto: 844.16; Class4 -> Label: 1390.08, Auto: 1421.28; 
Processed 00175.png: Class1 -> Label: 995.89, Auto: 589.40; Class2 -> Label: 1224.65, Auto: 1106.96; Class3 -> Label: 1205.75, Auto: 1025.19; Class4 -> Label: 1335.73, Auto: 614.45; 
Processed 00297.png: Class1 -> Label: 1460.98, Auto: 1273.81; Class2 -> Label: 973.67,

Processed 00088.png: Class1 -> Label: 1612.23, Auto: 1513.92; Class2 -> Label: 1030.86, Auto: 1062.53; Class3 -> Label: 1058.75, Auto: 1072.93; Class4 -> Label: 1504.94, Auto: 1490.76; 
Processed 00090.png: Class1 -> Label: 1676.51, Auto: 1577.73; Class2 -> Label: 937.75, Auto: 936.33; Class3 -> Label: 1032.28, Auto: 1039.84; Class4 -> Label: 1677.46, Auto: 1528.57; 
Processed 00352.png: Class1 -> Label: 1656.19, Auto: 1726.61; Class2 -> Label: 921.21, Auto: 902.30; Class3 -> Label: 884.81, Auto: 898.05; Class4 -> Label: 1611.76, Auto: 1616.01; 
Processed 00356.png: Class1 -> Label: 1531.41, Auto: 1497.38; Class2 -> Label: 970.36, Auto: 924.99; Class3 -> Label: 890.48, Auto: 921.68; Class4 -> Label: 1529.52, Auto: 1615.07; 
Processed 00079.png: Class1 -> Label: 1404.73, Auto: 1301.70; Class2 -> Label: 778.46, Auto: 817.70; Class3 -> Label: 729.31, Auto: 779.41; Class4 -> Label: 1404.26, Auto: 1422.22; 
Processed 00406.png: Class1 -> Label: 1227.49, Auto: 1063.95; Class2 -> Label: 824.3

Processed 00269.png: Class1 -> Label: 1297.91, Auto: 1287.99; Class2 -> Label: 1002.03, Auto: 19.38; Class3 -> Label: 1171.71, Auto: 944.37; Class4 -> Label: 1417.02, Auto: 1324.38; 
Processed 00119.png: Class1 -> Label: 1120.67, Auto: 1128.23; Class2 -> Label: 1113.58, Auto: 1136.27; Class3 -> Label: 1096.09, Auto: 1076.71; Class4 -> Label: 971.78, Auto: 766.18; 
Processed 00334.png: Class1 -> Label: 1245.45, Auto: 1240.25; Class2 -> Label: 885.76, Auto: 924.04; Class3 -> Label: 901.36, Auto: 949.57; Class4 -> Label: 1292.71, Auto: 1210.95; 
Processed 00199.png: Class1 -> Label: 1445.38, Auto: 1371.65; Class2 -> Label: 837.55, Auto: 789.81; Class3 -> Label: 871.58, Auto: 855.98; Class4 -> Label: 1647.21, Auto: 1533.77; 
Processed 00087.png: Class1 -> Label: 1777.66, Auto: 1728.03; Class2 -> Label: 1132.48, Auto: 1129.18; Class3 -> Label: 1140.05, Auto: 1173.13; Class4 -> Label: 2139.71, Auto: 2115.14; 
Processed 00121.png: Class1 -> Label: 991.63, Auto: 949.57; Class2 -> Label: 744.43

Processed 00385.png: Class1 -> Label: 2055.58, Auto: 1014.79; Class2 -> Label: 662.19, Auto: 799.73; Class3 -> Label: 914.12, Auto: 356.86; Class4 -> Label: 1807.91, Auto: 709.93; 
Processed 00391.png: Class1 -> Label: 1521.01, Auto: 1643.43; Class2 -> Label: 925.93, Auto: 881.03; Class3 -> Label: 872.05, Auto: 636.20; Class4 -> Label: 1461.93, Auto: 1482.72; 
Processed 00035.png: Class1 -> Label: 1064.89, Auto: 1000.14; Class2 -> Label: 783.19, Auto: 774.21; Class3 -> Label: 889.54, Auto: 848.89; Class4 -> Label: 980.29, Auto: 944.84; 
Processed 00036.png: Class1 -> Label: 911.28, Auto: 612.56; Class2 -> Label: 1046.93, Auto: 968.95; Class3 -> Label: 958.07, Auto: 931.61; Class4 -> Label: 1253.48, Auto: 1053.55; 
Processed 00133.png: Class1 -> Label: 829.04, Auto: 549.70; Class2 -> Label: 1085.22, Auto: 1088.53; Class3 -> Label: 943.89, Auto: 942.00; Class4 -> Label: 1389.61, Auto: 1479.41; 
Processed 00173.png: Class1 -> Label: 974.62, Auto: 494.87; Class2 -> Label: 1078.13, Auto: 11

In [2]:
results_long_df

,PID,Class,Auto Area,Manual Area
0,00261,1,1153.281250,1229.851562
1,00261,2,1147.136719,1147.136719
2,00261,3,1198.656250,1229.378906
3,00261,4,1246.394531,1206.218750
4,00212,1,1433.566406,1400.953125
...,...,...,...,...
1639,00091,4,1533.296875,1546.058594
1640,00377,1,661.718750,715.128906
1641,00377,2,770.429688,795.480469
1642,00377,3,805.406250,851.726562


In [3]:
manual_prompt_id = ['00000', '00001', '00002']
# 逻辑：根据 prompt_dir 中是否包含 "1" 或 "2" 来决定去除 manual_prompt_id 中的前几项
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]
results_df = filter_pid(results_long_df)
results_df

Remove prefixes: ['00000']


,PID,Class,Auto Area,Manual Area
0,00261,1,1153.281250,1229.851562
1,00261,2,1147.136719,1147.136719
2,00261,3,1198.656250,1229.378906
3,00261,4,1246.394531,1206.218750
4,00212,1,1433.566406,1400.953125
...,...,...,...,...
1639,00091,4,1533.296875,1546.058594
1640,00377,1,661.718750,715.128906
1641,00377,2,770.429688,795.480469
1642,00377,3,805.406250,851.726562


In [4]:
import numpy as np
import pandas as pd
import pingouin as pg
from scipy.stats import ttest_ind, ks_2samp, shapiro

# 假设 df_results 已经包含了分割面积计算结果，
# 并且包含 "PID", "Class", "Manual Area", "Auto Area" 等列

# ----------------- 统计学检验、MAPE 和 ICC -----------------
# 用于存储各类别的 MAPE 和 ICC 结果
mape_results = []  # 每项格式：[Class, MAPE, MAPE_std]
icc_results = []   # 每项格式：[Class, ICC, ICC_95%_CI_Lower, ICC_95%_CI_Upper]
# 用于存储每个类别的 p-value
p_value_dict = {}
df_results = results_df
# 对每个类别进行统计检验及误差、ICC 分析
for class_value in [1, 2, 3, 4]:
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    # 提取手动分割（Label）与自动分割（Auto Area）数值
    label_values = df_class["Manual Area"].values *100
    auto_values = df_class["Auto Area"].values*100

    # -------------- 正态性检验 --------------
    shapiro_label = shapiro(label_values)
    shapiro_auto_seg = shapiro(auto_values)

    label_is_normal = shapiro_label.pvalue >= 0.05
    auto_seg_is_normal = shapiro_auto_seg.pvalue >= 0.05

    print(f"\nClass {class_value}:")
    print(f"  Shapiro-Wilk Normality Test (Label): p-value = {shapiro_label.pvalue:.3f} -> {'Normal' if label_is_normal else 'Non-Normal'}")
    print(f"  Shapiro-Wilk Normality Test (Auto-Seg): p-value = {shapiro_auto_seg.pvalue:.3f} -> {'Normal' if auto_seg_is_normal else 'Non-Normal'}")

    # -------------- 选择统计检验方法 --------------
    if label_is_normal and auto_seg_is_normal:
        t_stat, p_value = ttest_ind(label_values, auto_values, equal_var=False)
        print("  Method: Independent Two-Sample T-Test")
        print(f"  T-statistic: {t_stat:.4f}, P-value: {p_value:.3f}")
    else:
        ks_stat, p_value = ks_2samp(label_values, auto_values)
        print("  Method: Kolmogorov-Smirnov Test")
        print(f"  KS-statistic: {ks_stat:.4f}, P-value: {p_value:.3f}")

    # 存储 p-value，用于后续汇总打印
    p_value_dict[class_value] = p_value

    # -------------- 解释检验结果 --------------
    if p_value < 0.05:
        print("  - Significant difference detected (p < 0.05).")
    else:
        print("  - No significant difference detected (p >= 0.05).")

    # -------------- 计算 MAPE --------------
    # 防止除 0，使用一个很小的 epsilon
    epsilon = 1e-6
    ape = np.abs((auto_values - label_values) / (label_values + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append([class_value, mape, mape_std])

    # -------------- 计算 ICC --------------
    df_long = df_class.melt(id_vars=["PID"], value_vars=["Manual Area", "Auto Area"],
                            var_name="Method", value_name="Area")
    icc_df = pg.intraclass_corr(data=df_long, targets="PID", raters="Method", ratings="Area")
    # 提取 ICC2 相关行
    icc_row = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
    icc_value = icc_row["ICC"]
    if "CI95%" in icc_row:
        ci_lower, ci_upper = icc_row["CI95%"]
    else:
        ci_lower, ci_upper = np.nan, np.nan
    icc_results.append([class_value, icc_value, ci_lower, ci_upper])

# ----------------- 显示 MAPE 和 ICC 结果 -----------------
mape_df = pd.DataFrame(mape_results, columns=['Class', 'MAPE', 'MAPE_std'])
icc_df = pd.DataFrame(icc_results, columns=['Class', 'ICC', 'ICC_95%_CI_Lower', 'ICC_95%_CI_Upper'])

# 将 MAPE 及其标准差转换为百分比格式，保留一位小数
mape_df['MAPE'] = (mape_df['MAPE'] * 100).round(1)
mape_df['MAPE_std'] = (mape_df['MAPE_std'] * 100).round(1)

mape_df = mape_df.sort_values(by="Class")
icc_df = icc_df.sort_values(by="Class")

# 打印 MAPE 结果，格式示例：14.9 ± 13.1%
print("\n### MAPE Results ###")
for _, row in mape_df.iterrows():
    print(f"Class {int(row['Class'])}: {row['MAPE']} ± {row['MAPE_std']}%")

# 打印 ICC 结果（包含 95% 置信区间）
print("\n### ICC Results ###")
for _, row in icc_df.iterrows():
    print(f"Class {int(row['Class'])}: {row['ICC']:.3f}, [{row['ICC_95%_CI_Lower']:.2f} - {row['ICC_95%_CI_Upper']:.2f}]")

# ----------------- 汇总打印 p-value -----------------
# 定义类别与对应的名称（根据需要调整）
class_label_mapping = {
    1: "Right LES",
    2: "Right MF",
    3: "Left MF",
    4: "Left LES"
}
print("\n### p-values Summary ###")
# 打印第一行：类别名称和对应的编号
header = " ".join([f"{class_label_mapping[cv]} {cv}" for cv in [1, 2, 3, 4]])
# 打印第二行：对应的 p-value，保留三位小数
p_values_line = " ".join([f"{p_value_dict.get(cv, np.nan):.3f}" for cv in [1, 2, 3, 4]])
print(header)
print(p_values_line)



Class 1:
  Shapiro-Wilk Normality Test (Label): p-value = 0.002 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.058 -> Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.2098, P-value: 0.000
  - Significant difference detected (p < 0.05).

Class 2:
  Shapiro-Wilk Normality Test (Label): p-value = 0.000 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.000 -> Non-Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.0585, P-value: 0.484
  - No significant difference detected (p >= 0.05).

Class 3:
  Shapiro-Wilk Normality Test (Label): p-value = 0.001 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.000 -> Non-Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.0683, P-value: 0.295
  - No significant difference detected (p >= 0.05).

Class 4:
  Shapiro-Wilk Normality Test (Label): p-value = 0.015 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.081 -> Normal
  Method: Kolmogorov-Smirnov Test